In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
from shapely.geometry import shape
from shapely.geometry import Polygon

In [ ]:
# Leer solo las columnas necesarias desde el archivo .parquet
df = pd.read_parquet('quejas-final.parquet', columns=['barrio_localización', 'tipo_solicitud', 'geo_shape'])
dfvul = pd.read_csv("vulnerabilidad-por-barrios.csv", sep=";", usecols=['Ind_Equip', 'Ind_Dem', 'Ind_Econom', 'Ind_Global', 'Nombre', 'Vul_Equip', 'Vul_Dem', 'Vul_Econom', 'Vul_Global'])
vulnerability_mapping = {
    'Vulnerabilidad Alta': 5,
    'Vulnerabilidad Media': 2.5,
    'Vulnerabilidad Baja': 1
}

# Aplicar el mapeo a la columna de vulnerabilidad para crear una nueva columna
dfvul['Nivel_Equip'] = dfvul['Vul_Equip'].map(vulnerability_mapping)
dfvul['Nivel_Dem'] = dfvul['Vul_Dem'].map(vulnerability_mapping)
dfvul['Nivel_Econom'] = dfvul['Vul_Econom'].map(vulnerability_mapping)
dfvul['Nivel_Global'] = dfvul['Vul_Global'].map(vulnerability_mapping)

# Procesar datos
conteo_barrio_tipo = df.groupby(['barrio_localización', 'tipo_solicitud']).size().reset_index(name='count')
pivot_conteo = conteo_barrio_tipo.pivot(index='barrio_localización', columns='tipo_solicitud', values='count').fillna(0).reset_index()
conteo_vul = pd.merge(pivot_conteo, dfvul, left_on="barrio_localización", right_on="Nombre", how="left")

# Filtrar columnas necesarias
dfvul_filtrado = pd.merge(conteo_vul, df[['barrio_localización', 'geo_shape']], on="barrio_localización", how="left")

# Convertir geo_shape a geometrías y simplificar las geometrías
def convert_and_simplify(geojson_str, tolerance=0.001):
    geom = shape(json.loads(geojson_str))
    if isinstance(geom, Polygon):
        return geom.simplify(tolerance)
    else:
        return geom

dfvul_filtrado['geometry'] = dfvul_filtrado['geo_shape'].apply(lambda x: convert_and_simplify(x))

# Crear un GeoDataFrame con las geometrías
gdf = gpd.GeoDataFrame(dfvul_filtrado, geometry='geometry')

# Crear un mapa base usando las geometrías
fig = make_subplots(
    rows=1, cols=1,
    subplot_titles=['List of vulnerabilities and requests by neighbourhood'],
    specs=[[{"type": "mapbox"}]]
)

# Añadir customdata para cada trazo
gdf['customdata_equip'] = gdf[['Ind_Equip','Vul_Equip', 'Síndic', 'Sugerencia', 'Queja', 'Otras', 'Defensor']].values.tolist()
gdf['customdata_dem'] = gdf[["Ind_Dem",'Vul_Dem', 'Síndic', 'Sugerencia', 'Queja', 'Otras', 'Defensor']].values.tolist()
gdf['customdata_econom'] = gdf[["Ind_Econom",'Vul_Econom', 'Síndic', 'Sugerencia', 'Queja', 'Otras', 'Defensor']].values.tolist()
gdf['customdata_global'] = gdf[["Ind_Global",'Vul_Global', 'Síndic', 'Sugerencia', 'Queja', 'Otras', 'Defensor']].values.tolist()

# Crear trazos de Choroplethmapbox para cada tipo de vulnerabilidad
equip_trace = go.Choroplethmapbox(
    geojson=gdf.set_index('barrio_localización')['geometry'].__geo_interface__,
    locations=gdf['barrio_localización'],
    z=gdf['Nivel_Equip'],
    colorscale='YlGnBu',
    colorbar_title="Vulnerability Equipment",
    marker_line_width=1,
    marker_opacity=0.7,
    visible=True,  # Solo el primer trazo es visible inicialmente
    customdata=gdf['customdata_equip'],
    hovertemplate='<b>Barrio:</b> %{location}<br>' +
                  '<b>Vulnerability Equipment:</b> %{customdata[0]}<br>' +
                  '<b>Nivel:</b> %{customdata[1]}<br>' +
                  '<b>Number of Sindic:</b> %{customdata[2]}<br>' +
                  '<b>Number of Suggestions:</b> %{customdata[3]}<br>' +
                  '<b>Number of Complaints:</b> %{customdata[4]}<br>' +
                  '<b>Number of Others:</b> %{customdata[5]}<br>' +
                  '<b>Number of Defender:</b> %{customdata[6]}<extra></extra>'
)

dem_trace = go.Choroplethmapbox(
    geojson=gdf.set_index('barrio_localización')['geometry'].__geo_interface__,
    locations=gdf['barrio_localización'],
    z=gdf['Nivel_Dem'],
    colorscale='YlGnBu',
    colorbar_title="Demographic Vulnerability",
    marker_line_width=1,
    marker_opacity=0.7,
    visible=False,  # Este trazo no es visible inicialmente
    customdata=gdf['customdata_dem'],
    hovertemplate='<b>Barrio:</b> %{location}<br>' +
                  '<b>Demographic Vulnerability:</b> %{customdata[0]}<br>' +
                  '<b>Nivel:</b> %{customdata[1]}<br>' +
                  '<b>Number of Sindic::</b> %{customdata[2]}<br>' +
                  '<b>Number of  Suggestions:</b> %{customdata[3]}<br>' +
                  '<b>Number of  Complaints:</b> %{customdata[4]}<br>' +
                  '<b>Number of  Others:</b> %{customdata[5]}<br>' +
                  '<b>Number of  Defender:</b> %{customdata[6]}<extra></extra>'
)


eco_trace = go.Choroplethmapbox(
    geojson=gdf.set_index('barrio_localización')['geometry'].__geo_interface__,
    locations=gdf['barrio_localización'],
    z=gdf['Nivel_Econom'],
    colorscale='YlGnBu',
    colorbar_title="Vulnerabilidad Económica",
    marker_line_width=1,
    marker_opacity=0.7,
    visible=False,  # Este trazo no es visible inicialmente
    customdata=gdf['customdata_econom'],
    hovertemplate='<b>Barrio:</b> %{location}<br>' +
                  '<b>Economic Vulnerability:</b> %{customdata[0]}<br>' +
                  '<b>Nivel:</b> %{customdata[1]}<br>' +
                  '<b>Number of Sindic::</b> %{customdata[2]}<br>' +
                  '<b>Number of  Suggestions:</b> %{customdata[3]}<br>' +
                  '<b>Number of  Complaints:</b> %{customdata[4]}<br>' +
                  '<b>Number of  Others:</b> %{customdata[5]}<br>' +
                  '<b>Number of  Defender:</b> %{customdata[6]}<extra></extra>'
)

glob_trace = go.Choroplethmapbox(
    geojson=gdf.set_index('barrio_localización')['geometry'].__geo_interface__,
    locations=gdf['barrio_localización'],
    z=gdf['Nivel_Global'],
    colorscale='YlGnBu',
    colorbar_title="Global Vulnerability",
    marker_line_width=1,
    marker_opacity=0.7,
    visible=False,  # Este trazo no es visible inicialmente
    customdata=gdf['customdata_global'],
    hovertemplate='<b>Barrio:</b> %{location}<br>' +
                  '<b>Global Vulnerability: </b> %{customdata[0]}<br>' +
                  '<b>Nivel:</b> %{customdata[1]}<br>' +
                  '<b>Number of Sindic::</b> %{customdata[2]}<br>' +
                  '<b>Number of  Suggestions:</b> %{customdata[3]}<br>' +
                  '<b>Number of  Complaints:</b> %{customdata[4]}<br>' +
                  '<b>Number of  Others:</b> %{customdata[5]}<br>' +
                  '<b>Number of  Defender:</b> %{customdata[6]}<extra></extra>'
)
# Añadir trazos a la figura
fig.add_trace(equip_trace, row=1, col=1)
fig.add_trace(dem_trace, row=1, col=1)
fig.add_trace(eco_trace, row=1, col=1)
fig.add_trace(glob_trace, row=1, col=1)

# Configuración del mapa de Valencia
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=11,
    mapbox_center={"lat": 39.4699, "lon": -0.3763},  # Coordenadas para centrar el mapa en Valencia
    title_text="Vulnerability and applications"
)

# Configuración del menú de actualización
fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "args": [{"visible": [True, False, False, False]}],
                    "label": "Equipment",
                    "method": "update"
                },
                {
                    "args": [{"visible": [False, True, False, False]}],
                    "label": "Demographic",
                    "method": "update"
                },
                {
                    "args": [{"visible": [False, False, True, False]}],
                    "label": "Economic",
                    "method": "update"
                },
                {
                    "args": [{"visible": [False, False, False, True]}],
                    "label": "Global",
                    "method": "update"
                }
            ],
            "direction": "down",
            "pad": {"r": 10, "t": 10},
            "showactive": True,
            "x": -0.3
            "xanchor": "left",
            "y": 1.2,
            "yanchor": "top"
        }
    ]
)

# Finalmente, asegúrate de mostrar la figura
fig.show()

In [ ]:
# Leer solo las columnas necesarias desde el archivo .parquet
df = pd.read_parquet('quejas-final.parquet', columns=['distrito_localización', 'tipo_solicitud', 'geo_shape'])
dfvul = pd.read_csv("vulnerabilidad-por-barrios.csv", sep=";", usecols=['Distrito', 'Ind_Equip', 'Ind_Dem', 'Ind_Econom', 'Ind_Global', 'Vul_Equip', 'Vul_Dem', 'Vul_Econom', 'Vul_Global'])

# Convertir las columnas de vulnerabilidad a valores numéricos usando el mapeo proporcionado
vulnerability_mapping = {
    'Vulnerabilidad Alta': 5,
    'Vulnerabilidad Media': 2.5,
    'Vulnerabilidad Baja': 1
}
dfvul['Nivel_Equip'] = dfvul['Vul_Equip'].map(vulnerability_mapping)
dfvul['Nivel_Dem'] = dfvul['Vul_Dem'].map(vulnerability_mapping)
dfvul['Nivel_Econom'] = dfvul['Vul_Econom'].map(vulnerability_mapping)
dfvul['Nivel_Global'] = dfvul['Vul_Global'].map(vulnerability_mapping)

# Agrupar por distrito y calcular la media de los índices de vulnerabilidad y tomar la primera ocurrencia de las categorías
dfvul_agg = dfvul.groupby('Distrito').agg({
    'Ind_Equip': 'mean',
    'Ind_Dem': 'mean',
    'Ind_Econom': 'mean',
    'Ind_Global': 'mean',
    'Vul_Equip': 'first',
    'Vul_Dem': 'first',
    'Vul_Econom': 'first',
    'Vul_Global': 'first'
}).reset_index()

# Procesar datos de solicitudes
conteo_distrito_tipo = df.groupby(['distrito_localización', 'tipo_solicitud']).size().reset_index(name='count')
pivot_conteo = conteo_distrito_tipo.pivot(index='distrito_localización', columns='tipo_solicitud', values='count').fillna(0).reset_index()
conteo_vul = pd.merge(pivot_conteo, dfvul_agg, left_on="distrito_localización", right_on="Distrito", how="left")

# Filtrar columnas necesarias
dfvul_filtrado = pd.merge(conteo_vul, df[['distrito_localización', 'geo_shape']], on="distrito_localización", how="left")

# Convertir geo_shape a geometrías y simplificar las geometrías
def convert_and_simplify(geojson_str, tolerance=0.001):
    geom = shape(json.loads(geojson_str))
    if isinstance(geom, Polygon):
        return geom.simplify(tolerance)
    else:
        return geom

dfvul_filtrado['geometry'] = dfvul_filtrado['geo_shape'].apply(lambda x: convert_and_simplify(x))

# Crear un GeoDataFrame con las geometrías
gdf = gpd.GeoDataFrame(dfvul_filtrado, geometry='geometry')

# Crear un mapa base usando las geometrías
fig = make_subplots(
    rows=1, cols=1,
    subplot_titles=['Vulnerability and applications'],
    specs=[[{"type": "mapbox"}]]
)

# Añadir customdata para cada trazo
gdf['customdata_equip'] = gdf[['Ind_Equip', 'Vul_Equip', 'Síndic', 'Sugerencia', 'Queja', 'Otras', 'Defensor']].values.tolist()
gdf['customdata_dem'] = gdf[['Ind_Dem', 'Vul_Dem', 'Síndic', 'Sugerencia', 'Queja', 'Otras', 'Defensor']].values.tolist()
gdf['customdata_econom'] = gdf[['Ind_Econom', 'Vul_Econom', 'Síndic', 'Sugerencia', 'Queja', 'Otras', 'Defensor']].values.tolist()
gdf['customdata_global'] = gdf[['Ind_Global', 'Vul_Global', 'Síndic', 'Sugerencia', 'Queja', 'Otras', 'Defensor']].values.tolist()

# Crear trazos de Choroplethmapbox para cada tipo de vulnerabilidad
equip_trace = go.Choroplethmapbox(
    geojson=gdf.set_index('distrito_localización')['geometry'].__geo_interface__,
    locations=gdf['distrito_localización'],
    z=gdf['Ind_Equip'],
    colorscale='YlGnBu',
    colorbar_title="Vulnerability Equipment",
    marker_line_width=1,
    marker_opacity=0.7,
    visible=True,  # Solo el primer trazo es visible inicialmente
    customdata=gdf['customdata_equip'],
    hovertemplate='<b>Distrito:</b> %{location}<br>' +
                  '<b>Vulnerability Equipment:</b> %{customdata[0]}<br>' +
                  '<b>Nivel:</b> %{customdata[1]}<br>' +
                  '<b>Number of Sindic::</b> %{customdata[2]}<br>' +
                  '<b>Number of  Suggestions:</b> %{customdata[3]}<br>' +
                  '<b>Number of  Complaints:</b> %{customdata[4]}<br>' +
                  '<b>Number of  Others:</b> %{customdata[5]}<br>' +
                  '<b>Number of  Defender:</b> %{customdata[6]}<extra></extra>'
)

dem_trace = go.Choroplethmapbox(
    geojson=gdf.set_index('distrito_localización')['geometry'].__geo_interface__,
    locations=gdf['distrito_localización'],
    z=gdf['Ind_Dem'],
    colorscale='YlGnBu',
    colorbar_title="Demographic Vulnerability",
    marker_line_width=1,
    marker_opacity=0.7,
    visible=False,  # Este trazo no es visible inicialmente
    customdata=gdf['customdata_dem'],
    hovertemplate='<b>Distrito:</b> %{location}<br>' +
                  '<b>Demographic Vulnerability:</b> %{customdata[0]}<br>' +
                  '<b>Nivel:</b> %{customdata[1]}<br>' +
                  '<b>Number of Sindic::</b> %{customdata[2]}<br>' +
                  '<b>Number of  Suggestions:</b> %{customdata[3]}<br>' +
                  '<b>Number of  Complaints:</b> %{customdata[4]}<br>' +
                  '<b>Number of  Others:</b> %{customdata[5]}<br>' +
                  '<b>Number of  Defender:</b> %{customdata[6]}<extra></extra>'
)

eco_trace = go.Choroplethmapbox(
    geojson=gdf.set_index('distrito_localización')['geometry'].__geo_interface__,
        locations=gdf['distrito_localización'],
    z=gdf['Ind_Econom'],
    colorscale='YlGnBu',
    colorbar_title="Economic Vulnerability",
    marker_line_width=1,
    marker_opacity=0.7,
    visible=False,  # Este trazo no es visible inicialmente
    customdata=gdf['customdata_econom'],
    hovertemplate='<b>Distrito:</b> %{location}<br>' +
                  '<b>Economic Vulnerability:</b> %{customdata[0]}<br>' +
                  '<b>Nivel:</b> %{customdata[1]}<br>' +
                  '<b>Number of Sindic::</b> %{customdata[2]}<br>' +
                  '<b>Number of  Suggestions:</b> %{customdata[3]}<br>' +
                  '<b>Number of  Complaints:</b> %{customdata[4]}<br>' +
                  '<b>Number of  Others:</b> %{customdata[5]}<br>' +
                  '<b>Number of  Defender:</b> %{customdata[6]}<extra></extra>'
)

glob_trace = go.Choroplethmapbox(
    geojson=gdf.set_index('distrito_localización')['geometry'].__geo_interface__,
    locations=gdf['distrito_localización'],
    z=gdf['Ind_Global'],
    colorscale='YlGnBu',
    colorbar_title="Global Vulnerability",
    marker_line_width=1,
    marker_opacity=0.7,
    visible=False,  # Este trazo no es visible inicialmente
    customdata=gdf['customdata_global'],
    hovertemplate='<b>Distrito:</b> %{location}<br>' +
                  '<b>Global Vulnerability:</b> %{customdata[0]}<br>' +
                  '<b>Nivel:</b> %{customdata[1]}<br>' +
                  '<b>Number of Sindic::</b> %{customdata[2]}<br>' +
                  '<b>Number of  Suggestions:</b> %{customdata[3]}<br>' +
                  '<b>Number of  Complaints:</b> %{customdata[4]}<br>' +
                  '<b>Number of  Others:</b> %{customdata[5]}<br>' +
                  '<b>Number of  Defender:</b> %{customdata[6]}<extra></extra>'
)

# Añadir trazos a la figura
fig.add_trace(equip_trace, row=1, col=1)
fig.add_trace(dem_trace, row=1, col=1)
fig.add_trace(eco_trace, row=1, col=1)
fig.add_trace(glob_trace, row=1, col=1)

# Configuración del mapa de Valencia
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=11,
    mapbox_center={"lat": 39.4699, "lon": -0.3763},  # Coordenadas para centrar el mapa en Valencia
    title_text="Vulnerability and applications"
)

# Configuración del menú de actualización
fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "args": [{"visible": [True, False, False, False]}],
                    "label": "Equipment",
                    "method": "update"
                },
                {
                    "args": [{"visible": [False, True, False, False]}],
                    "label": "Demographic",
                    "method": "update"
                },
                {
                    "args": [{"visible": [False, False, True, False]}],
                    "label": "Economic",
                    "method": "update"
                },
                {
                    "args": [{"visible": [False, False, False, True]}],
                    "label": "Global",
                    "method": "update"
                }
            ],
            "direction": "down",
            "pad": {"r": 10, "t": 10},
            "showactive": True,
            "x": 0,
            "xanchor": "left",
            "y": 1.2,
            "yanchor": "top"
        }
    ]
)

# Finalmente, asegúrate de mostrar la figura
fig.show()